# Level 7: Advanced MLOps - Uygulamalar

Bu notebook, ileri seviye MLOps konularını pratik örneklerle açıklamaktadır.

**Not:** Başlamadan önce bu seviyenin gereksinimlerini kurduğunuzdan emin olun:
```bash
pip install -r requirements.txt
```

## 1. Multi-Model Orchestration (Çoklu Model Orkestrasyonu)

Bu örnekte, `Ray Serve` kullanarak gelen isteklere göre farklı modellere yönlendirme yapan basit bir yapı kuracağız.

In [ ]:
import ray
from ray import serve
from starlette.requests import Request

# Ray'i başlat (eğer zaten çalışmıyorsa)
if not ray.is_initialized():
    ray.init(num_cpus=4)

# Serve'i başlat
serve.start()

@serve.deployment(name="model_a")
class ModelA:
    def predict(self, data):
        return {"model": "A", "prediction": data * 2}

@serve.deployment(name="model_b")
class ModelB:
    def predict(self, data):
        return {"model": "B", "prediction": data + 5}

@serve.deployment(route_prefix="/invocations")
class Router:
    def __init__(self, model_a_handle, model_b_handle):
        self.model_a = model_a_handle
        self.model_b = model_b_handle

    async def __call__(self, request: Request):
        data = await request.json()
        input_data = data['input']

        # Basit yönlendirme mantığı: Girdi 10'dan büyükse Model B'yi kullan
        if input_data > 10:
            result = await self.model_b.predict.remote(input_data)
        else:
            result = await self.model_a.predict.remote(input_data)
        
        return result

ModelA.deploy()
ModelB.deploy()
Router.deploy(ModelA.get_handle(), ModelB.get_handle())

print("Modeller ve yönlendirici dağıtıldı.")
print("Test etmek için yeni bir terminalde şu komutları kullanabilirsiniz:")
print("curl -X POST -H 'Content-Type: application/json' -d '{\"input\": 5}' http://localhost:8000/invocations")
print("curl -X POST -H 'Content-Type: application/json' -d '{\"input\": 15}' http://localhost:8000/invocations")

## 2. A/B Testing

İki modelin (A ve B) dönüşüm oranlarını karşılaştırmak için basit bir istatistiksel test (Z-testi) yapalım.

In [ ]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

# Model A'nın sonuçları
visitors_A = 1000
conversions_A = 100  # %10 dönüşüm oranı

# Model B'nin sonuçları
visitors_B = 1050
conversions_B = 125  # ~%11.9 dönüşüm oranı

# Z-testi uygulama
count = np.array([conversions_A, conversions_B])
nobs = np.array([visitors_A, visitors_B])

stat, p_value = proportions_ztest(count, nobs)

print(f"Z-istatistiği: {stat:.4f}")
print(f"P-değeri: {p_value:.4f}")

alpha = 0.05  # Anlamlılık seviyesi
if p_value < alpha:
    print("Sonuç istatistiksel olarak anlamlı: Model B, Model A'dan daha iyi performans gösteriyor.")
else:
    print("Sonuç istatistiksel olarak anlamlı değil: İki model arasında anlamlı bir fark yoktur.")

## 3. Feature Store Management (Özellik Deposu Yönetimi)

`Feast` kullanarak basit bir yerel özellik deposu oluşturalım.

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import os

# 1. Adım: Örnek verileri ve Feast deposu yapısını oluştur
if not os.path.exists('feature_repo/data'):
    os.makedirs('feature_repo/data')

user_data = pd.DataFrame({
    "event_timestamp": [pd.to_datetime(datetime.now() - timedelta(days=i)) for i in range(5)],
    "user_id": [101, 102, 101, 103, 102],
    "daily_transactions": [5, 8, 3, 1, 9],
    "avg_spend": [50.0, 120.5, 45.0, 20.0, 150.8]
})
user_data_path = "feature_repo/data/user_data.parquet"
user_data.to_parquet(user_data_path)

print(f"Örnek veri '{user_data_path}' dosyasına kaydedildi.")

# `feast init` komutu normalde bir CLI'dan çalıştırılır.
# Burada manuel olarak gerekli dosyaları oluşturacağız.
feature_repo_py = """
from feast import Entity, Feature, FeatureView, FileSource, ValueType
from datetime import timedelta

user_data = FileSource(
    path="data/user_data.parquet",
    event_timestamp_column="event_timestamp",
)

user = Entity(name="user_id", value_type=ValueType.INT64, description="User ID")

user_features_view = FeatureView(
    name="user_features",
    entities=["user_id"],
    ttl=timedelta(days=1),
    features=[
        Feature(name="daily_transactions", dtype=ValueType.INT64),
        Feature(name="avg_spend", dtype=ValueType.FLOAT),
    ],
    online=True,
    source=user_data,
    tags={},
)
"""
with open("feature_repo/example.py", "w") as f:
    f.write(feature_repo_py)

print("Feast tanım dosyası 'feature_repo/example.py' oluşturuldu.")

In [ ]:
# 2. Adım: Feast deposunu uygula ve özellik al

# Normalde terminalde `feast apply` ve `feast materialize` çalıştırılır.
# Notebook içinde FeatureStore nesnesini kullanarak bu işlemleri simüle edebiliriz.

from feast import FeatureStore

try:
    # Depoyu kur
    store = FeatureStore(repo_path="./feature_repo")

    print("\n--- Online Store'dan Özellik Alma ---")
    # Özellikleri almak için bir varlık DataFrame'i oluştur
    entity_df = pd.DataFrame.from_dict({
        "user_id": [101, 102, 104], # 104 id'li kullanıcı veride yok
        "event_timestamp": [datetime.now()] * 3
    })

    # Online store'dan en güncel özellikleri al
    # Bunun için önce `feast materialize` komutuyla verinin online store'a yüklenmesi gerekir.
    # Bu notebook'ta sadece offline alımı gösteriyoruz.

    print("\n--- Offline Store'dan Geçmiş Özellikleri Alma ---")
    # Model eğitimi için geçmiş özellikleri al
    training_df = store.get_historical_features(
        entity_df=entity_df,
        feature_refs=[
            'user_features:daily_transactions',
            'user_features:avg_spend'
        ],
    ).to_df()

    print("Eğitim için alınan özellikler:")
    print(training_df)

except Exception as e:
    print(f"Bir hata oluştu: {e}")
    print("Lütfen 'feature_repo' dizininde `feast apply` ve `feast materialize-incremental 2024-01-01T00:00:00` komutlarını çalıştırarak deneyin.")

## 4. MLOps Best Practices (MLOps İyi Uygulamaları)

Bu bölüm, koddan ziyade kavramsal bir özettir. `Advanced_MLOps_Kavramlar.md` dosyasında detayları bulabilirsiniz.

- **Her Şey Kod Olarak (Everything as Code):**
  - **Altyapı (IaC):** `Terraform`, `Pulumi`
  - **Pipeline'lar (PaC):** `Airflow`, `Prefect`, `Kubeflow Pipelines` DAG'leri
  - **Veri ve Modeller:** `DVC` ile versiyonlama

- **CI/CD/CT:**
  - **CI (Sürekli Entegrasyon):** Kod değişikliklerinde otomatik test ve linting (örn: GitHub Actions, Jenkins).
  - **CD (Sürekli Dağıtım):** Testleri geçen modellerin otomatik olarak dağıtılması (örn: `Seldon`, `Kserve` ile entegrasyon).
  - **CT (Sürekli Eğitim):** Veri veya model performansında bozulma tespit edildiğinde modelin otomatik yeniden eğitilmesi.

- **İzlenebilirlik ve Gözlemlenebilirlik:**
  - **Veri ve Model Kalitesi:** `Evidently`, `WhyLogs`, `Great Expectations`
  - **Sistem Performansı:** `Prometheus`, `Grafana`

Bu araçlar ve pratikler, MLOps yaşam döngüsünü otomatikleştirmek, güvenilir kılmak ve ölçeklendirmek için temel oluşturur.